In [26]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve



In [13]:
!pip install xgboost

     -------------------------------------- 124.9/124.9 MB 6.5 MB/s eta 0:00:00


In [21]:
df_1 = pd.read_csv("morocco.csv")
# Sample 100,000 rows
df = df_1.sample(n=100000, random_state=42)
df.head()

,Unnamed: 0,acq_date,latitude,longitude,is_holiday,day_of_week,day_of_year,is_weekend,NDVI,SoilMoisture,...,wind_gust_quarterly_mean,dew_point_quarterly_mean,average_temperature_yearly_mean,maximum_temperature_yearly_mean,minimum_temperature_yearly_mean,precipitation_yearly_mean,snow_depth_yearly_mean,wind_gust_yearly_mean,dew_point_yearly_mean,is_fire
710465,710465,2015-01-23,34.591408,-2.343653,0.0,4.0,23.0,0.0,2071.0,40.0,...,768.53480,48.344566,65.561370,78.095894,54.025204,0.306986,999.9,877.81647,49.801643,1.0
628007,628007,2018-03-12,33.911026,-5.495805,0.0,0.0,71.0,0.0,6013.0,49.0,...,957.65546,41.272827,66.815480,80.186850,54.799725,1.941233,999.9,949.30770,48.057945,1.0
280977,280977,2022-11-24,31.678123,-4.733578,0.0,3.0,328.0,0.0,1382.0,11.0,...,695.23914,34.810870,71.861916,83.816850,59.490410,0.007699,999.9,806.69070,28.587397,0.0
345642,345642,2015-04-02,34.592340,-2.347825,0.0,3.0,92.0,0.0,3008.0,21.0,...,817.34220,41.308887,65.561370,78.095894,54.025204,0.306986,999.9,877.81647,49.801643,1.0
138561,138561,2021-03-13,27.534670,-9.859253,0.0,5.0,72.0,1.0,1066.0,4.0,...,989.29346,50.739132,69.074860,76.311200,60.608470,0.004590,999.9,984.04920,53.070217,0.0


In [22]:
df_train=df[df.acq_date<'2022-01-01']
df_valid=df[df.acq_date>='2022-01-01']

In [23]:
# Balance the training and validation datasets by taking the same number of positive and negative samples
def balance_dataset(df, target_column):
    # Get the minimum number of samples in each class
    min_samples = df[target_column].value_counts().min()
    
    # Balance the dataset
    df_balanced = df.groupby(target_column).apply(lambda x: x.sample(min_samples)).reset_index(drop=True)
    
    return df_balanced

# Balance training and validation datasets
df_train_balanced = balance_dataset(df_train, 'is_fire')
df_valid_balanced = balance_dataset(df_valid, 'is_fire')

# Shuffle the balanced datasets
df_train_balanced = df_train_balanced.sample(frac=1).reset_index(drop=True)
df_valid_balanced = df_valid_balanced.sample(frac=1).reset_index(drop=True)

# Remove 'acq_date' column
df_train_balanced.drop(columns='acq_date', inplace=True)
df_valid_balanced.drop(columns='acq_date', inplace=True)

# Convert all values to float32
df_train_balanced = df_train_balanced.astype('float32')
df_valid_balanced = df_valid_balanced.astype('float32')


C:\Users\Idriss\AppData\Local\Temp\ipykernel_42712\4275945084.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby(target_column).apply(lambda x: x.sample(min_samples)).reset_index(drop=True)
C:\Users\Idriss\AppData\Local\Temp\ipykernel_42712\4275945084.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby(target_column).apply(lambda x: x.sample(min_samples)).r

In [24]:
# Split the training dataset into features and target
X_train = df_train_balanced.drop(columns=['is_fire'])
y_train = df_train_balanced['is_fire']

# Split the validation dataset into features and target
X_valid = df_valid_balanced.drop(columns=['is_fire'])
y_valid = df_valid_balanced['is_fire']

# Initialize a StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training features and transform both the training and validation features
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

In [25]:
models = [
    ("Logistic Regression", LogisticRegression()),
    ("Random Forest", RandomForestClassifier()),
]

# Evaluate each model
results = []
for name, model in models:
    print(f"Evaluating {name}")

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Make predictions on the validation data
    y_pred = model.predict(X_valid)

    # Calculate accuracy and AUC
    accuracy = accuracy_score(y_valid, y_pred)
    y_proba = model.predict_proba(X_valid)
    auc = roc_auc_score(y_valid, y_proba[:, 1])

    # Calculate precision-recall curve
    precisions, recalls, _ = precision_recall_curve(y_valid, y_proba[:, 1])

    # Store the results
    results.append((name, accuracy, auc, precisions.mean(), recalls.mean()))

    # Print the results for this model
    print(f"{name}: Accuracy={accuracy:.4f}, AUC={auc:.4f}, Precision={precisions.mean():.4f}, Recall={recalls.mean():.4f}")

# Create a DataFrame to display the results
results_df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'AUC', 'Average Precision', 'Average Recall'])
print(results_df)

Evaluating Logistic Regression


C:\Users\Idriss\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression: Accuracy=0.8373, AUC=0.9344, Precision=0.8092, Recall=0.7172
Evaluating Random Forest
Random Forest: Accuracy=0.6122, AUC=0.9376, Precision=0.8222, Recall=0.5898
                 Model  Accuracy       AUC  Average Precision  Average Recall
0  Logistic Regression  0.837261  0.934436           0.809166        0.717165
1        Random Forest  0.612175  0.937553           0.822155        0.589758


In [27]:

xgb_model = xgb.XGBClassifier(n_jobs=1)
xgb_model.fit(X_train, y_train)

# Make predictions on the validation data
y_pred_xgb = xgb_model.predict(X_valid)
y_proba_xgb = xgb_model.predict_proba(X_valid)

# Calculate accuracy and AUC
accuracy_xgb = accuracy_score(y_valid, y_pred_xgb)
auc_xgb = roc_auc_score(y_valid, y_proba_xgb[:, 1])

# Calculate precision-recall curve
precisions_xgb, recalls_xgb, _ = precision_recall_curve(y_valid, y_proba_xgb[:, 1])

# Create a new DataFrame with the results
new_results = pd.DataFrame([{
    "Model": "XGBoost",
    "Accuracy": accuracy_xgb,
    "AUC": auc_xgb,
    "Average Precision": precisions_xgb.mean(),
    "Average Recall": recalls_xgb.mean()
}])

# Concatenate the new results to the existing results_df
results_df = pd.concat([results_df, new_results], ignore_index=True)

In [28]:
print(results_df)

                 Model  Accuracy       AUC  Average Precision  Average Recall
0  Logistic Regression  0.837261  0.934436           0.809166        0.717165
1        Random Forest  0.612175  0.937553           0.822155        0.589758
2              XGBoost  0.749877  0.929739           0.706266        0.885898


In [ ]:
import os
from itertools import product
from random import shuffle
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, Activation, LeakyReLU
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

# Assuming X_train_scaled, y_train, X_valid_scaled, y_valid are already defined

# Define the range of configurations
layers_range = range(3, 7)
units_range = [16, 32, 64, 128, 256, 300]
dropout_range = [0.6, 0.5, 0.4, 0.3, 0.0]
activation_functions = ['relu', 'gelu', 'softplus', 'leakyrelu']

# Generate all combinations
configurations = list(product(layers_range, units_range, dropout_range, activation_functions))

# Shuffle the configurations
shuffle(configurations)

# Initialize a DataFrame to store the results
dl_results = pd.DataFrame(columns=['layers', 'units', 'dropout', 'activation_function', 'accuracy', 'auc_pr', 'precision', 'recall'])

# Iterate through each configuration and train the model
for layers, units, dropout, activation in configurations:
    print(f"Training model with {layers} layers, {units} units, {dropout} dropout, and {activation} activation...")

    # Build the model
    model = Sequential()
    model.add(Input(shape=(X_train_scaled.shape[1],)))

    # Add the first layer separately to include input_shape
    model.add(Dense(units))
    if activation == 'leakyrelu':
        model.add(LeakyReLU(alpha=0.01))
    else:
        model.add(Activation(activation))
    model.add(Dropout(dropout))

    # Add additional layers
    for _ in range(1, layers):
        model.add(Dense(units))
        if activation == 'leakyrelu':
            model.add(LeakyReLU(alpha=0.01))
        else:
            model.add(Activation(activation))
        model.add(Dropout(dropout))

    # Add the output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', AUC(curve='PR', name='auc_pr'), Precision(name='precision'), Recall(name='recall')])

    # Define the checkpoint callback to save the best model weights only
    temp_checkpoint_path = f"temp_model_layers{layers}_units{units}_dropout{dropout}_{activation}.h5"
    checkpoint = ModelCheckpoint(temp_checkpoint_path, save_best_only=True, save_weights_only=True, monitor='val_accuracy', mode='max')

    # Train the model
    history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=512, validation_data=(X_valid_scaled, y_valid), callbacks=[checkpoint], verbose=0)

    # Extract the best metrics from the training history
    best_index = np.argmax(history.history['val_accuracy'])
    best_accuracy = history.history['val_accuracy'][best_index]
    best_auc_pr = history.history['val_auc_pr'][best_index]
    best_precision = history.history['val_precision'][best_index]
    best_recall = history.history['val_recall'][best_index]

    # Store the results in the DataFrame
    current_result = pd.DataFrame([[layers, units, dropout, activation, best_accuracy, best_auc_pr, best_precision, best_recall]],
                                  columns=['layers', 'units', 'dropout', 'activation_function', 'accuracy', 'auc_pr', 'precision', 'recall'])
    dl_results = pd.concat([dl_results, current_result], ignore_index=True)

    # Print the current model's result
    print(f"Results for model - Layers: {layers}, Units: {units}, Dropout: {dropout}, Activation: {activation}")
    print(f"Accuracy: {best_accuracy:.4f}, AUC-PR: {best_auc_pr:.4f}, Precision: {best_precision:.4f}, Recall: {best_recall:.4f}\n")


Training model with 3 layers, 256 units, 0.6 dropout, and relu activation...


C:\Users\Idriss\AppData\Local\Temp\ipykernel_42712\3867126322.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dl_results = pd.concat([dl_results, current_result], ignore_index=True)


Results for model - Layers: 3, Units: 256, Dropout: 0.6, Activation: relu
Accuracy: 0.8567, AUC-PR: 0.8991, Precision: 0.8088, Recall: 0.9342

Training model with 5 layers, 128 units, 0.5 dropout, and gelu activation...
Results for model - Layers: 5, Units: 128, Dropout: 0.5, Activation: gelu
Accuracy: 0.8449, AUC-PR: 0.8186, Precision: 0.8037, Recall: 0.9126

Training model with 6 layers, 128 units, 0.5 dropout, and relu activation...
Results for model - Layers: 6, Units: 128, Dropout: 0.5, Activation: relu
Accuracy: 0.8355, AUC-PR: 0.8985, Precision: 0.7757, Recall: 0.9440

Training model with 6 layers, 16 units, 0.5 dropout, and leakyrelu activation...
Results for model - Layers: 6, Units: 16, Dropout: 0.5, Activation: leakyrelu
Accuracy: 0.8886, AUC-PR: 0.9364, Precision: 0.9583, Recall: 0.8125

Training model with 5 layers, 16 units, 0.3 dropout, and softplus activation...
Results for model - Layers: 5, Units: 16, Dropout: 0.3, Activation: softplus
Accuracy: 0.8449, AUC-PR: 0.9556

Results for model - Layers: 3, Units: 64, Dropout: 0.6, Activation: relu
Accuracy: 0.8483, AUC-PR: 0.9195, Precision: 0.7955, Recall: 0.9377

Training model with 5 layers, 128 units, 0.0 dropout, and gelu activation...
Results for model - Layers: 5, Units: 128, Dropout: 0.0, Activation: gelu
Accuracy: 0.8488, AUC-PR: 0.9269, Precision: 0.7999, Recall: 0.9303

Training model with 3 layers, 256 units, 0.6 dropout, and leakyrelu activation...
Results for model - Layers: 3, Units: 256, Dropout: 0.6, Activation: leakyrelu
Accuracy: 0.8370, AUC-PR: 0.9233, Precision: 0.7817, Recall: 0.9352

Training model with 6 layers, 300 units, 0.5 dropout, and softplus activation...
Results for model - Layers: 6, Units: 300, Dropout: 0.5, Activation: softplus
Accuracy: 0.8544, AUC-PR: 0.9497, Precision: 0.8091, Recall: 0.9278

Training model with 6 layers, 32 units, 0.4 dropout, and leakyrelu activation...
Results for model - Layers: 6, Units: 32, Dropout: 0.4, Activation: leakyrelu
Accuracy: 0.8387, AUC

Results for model - Layers: 3, Units: 256, Dropout: 0.5, Activation: leakyrelu
Accuracy: 0.8562, AUC-PR: 0.8509, Precision: 0.8235, Recall: 0.9067

Training model with 6 layers, 256 units, 0.5 dropout, and softplus activation...
Results for model - Layers: 6, Units: 256, Dropout: 0.5, Activation: softplus
Accuracy: 0.8684, AUC-PR: 0.9350, Precision: 0.8296, Recall: 0.9273

Training model with 3 layers, 128 units, 0.4 dropout, and relu activation...
Results for model - Layers: 3, Units: 128, Dropout: 0.4, Activation: relu
Accuracy: 0.8692, AUC-PR: 0.9194, Precision: 0.8333, Recall: 0.9229

Training model with 6 layers, 16 units, 0.6 dropout, and gelu activation...
Results for model - Layers: 6, Units: 16, Dropout: 0.6, Activation: gelu
Accuracy: 0.8400, AUC-PR: 0.8693, Precision: 0.7837, Recall: 0.9391

Training model with 5 layers, 256 units, 0.4 dropout, and leakyrelu activation...
Results for model - Layers: 5, Units: 256, Dropout: 0.4, Activation: leakyrelu
Accuracy: 0.8692, AUC-PR:

...

Results for model - Layers: 3, Units: 64, Dropout: 0.4, Activation: leakyrelu
Accuracy: 0.8481, AUC-PR: 0.9265, Precision: 0.8056, Recall: 0.9175

Training model with 4 layers, 300 units, 0.3 dropout, and gelu activation...
Results for model - Layers: 4, Units: 300, Dropout: 0.3, Activation: gelu
Accuracy: 0.8822, AUC-PR: 0.8627, Precision: 0.8811, Recall: 0.8837

Training model with 6 layers, 300 units, 0.3 dropout, and leakyrelu activation...
Results for model - Layers: 6, Units: 300, Dropout: 0.3, Activation: leakyrelu
Accuracy: 0.8657, AUC-PR: 0.9436, Precision: 0.8256, Recall: 0.9273

Training model with 5 layers, 300 units, 0.0 dropout, and gelu activation...
Results for model - Layers: 5, Units: 300, Dropout: 0.0, Activation: gelu
Accuracy: 0.8667, AUC-PR: 0.8794, Precision: 0.8474, Recall: 0.8945

Training model with 5 layers, 300 units, 0.5 dropout, and relu activation...
Results for model - Layers: 5, Units: 300, Dropout: 0.5, Activation: relu
Accuracy: 0.8702, AUC-PR: 0.9077

Results for model - Layers: 3, Units: 32, Dropout: 0.3, Activation: leakyrelu
Accuracy: 0.8601, AUC-PR: 0.9055, Precision: 0.8219, Recall: 0.9195

Training model with 6 layers, 128 units, 0.5 dropout, and leakyrelu activation...
Results for model - Layers: 6, Units: 128, Dropout: 0.5, Activation: leakyrelu
Accuracy: 0.8459, AUC-PR: 0.9129, Precision: 0.7922, Recall: 0.9377

Training model with 3 layers, 32 units, 0.4 dropout, and softplus activation...
Results for model - Layers: 3, Units: 32, Dropout: 0.4, Activation: softplus
Accuracy: 0.8441, AUC-PR: 0.9401, Precision: 0.8027, Recall: 0.9126

Training model with 4 layers, 300 units, 0.6 dropout, and relu activation...
Results for model - Layers: 4, Units: 300, Dropout: 0.6, Activation: relu
Accuracy: 0.8382, AUC-PR: 0.9134, Precision: 0.7927, Recall: 0.9161

Training model with 3 layers, 16 units, 0.3 dropout, and softplus activation...
Results for model - Layers: 3, Units: 16, Dropout: 0.3, Activation: softplus
Accuracy: 0.8424, AU

Results for model - Layers: 6, Units: 64, Dropout: 0.3, Activation: leakyrelu
Accuracy: 0.8677, AUC-PR: 0.9175, Precision: 0.8182, Recall: 0.9455

Training model with 5 layers, 128 units, 0.5 dropout, and softplus activation...
Results for model - Layers: 5, Units: 128, Dropout: 0.5, Activation: softplus
Accuracy: 0.8461, AUC-PR: 0.9345, Precision: 0.8010, Recall: 0.9210

Training model with 4 layers, 32 units, 0.6 dropout, and gelu activation...
Results for model - Layers: 4, Units: 32, Dropout: 0.6, Activation: gelu
Accuracy: 0.8397, AUC-PR: 0.9469, Precision: 0.7820, Recall: 0.9421

Training model with 6 layers, 32 units, 0.3 dropout, and relu activation...
Results for model - Layers: 6, Units: 32, Dropout: 0.3, Activation: relu
Accuracy: 0.8427, AUC-PR: 0.9480, Precision: 0.7918, Recall: 0.9298

Training model with 3 layers, 256 units, 0.6 dropout, and softplus activation...
Results for model - Layers: 3, Units: 256, Dropout: 0.6, Activation: softplus
Accuracy: 0.8439, AUC-PR: 0.93

Results for model - Layers: 6, Units: 128, Dropout: 0.0, Activation: leakyrelu
Accuracy: 0.8648, AUC-PR: 0.9508, Precision: 0.8341, Recall: 0.9107

Training model with 6 layers, 64 units, 0.6 dropout, and softplus activation...
Results for model - Layers: 6, Units: 64, Dropout: 0.6, Activation: softplus
Accuracy: 0.8432, AUC-PR: 0.9337, Precision: 0.7862, Recall: 0.9426

Training model with 6 layers, 256 units, 0.6 dropout, and leakyrelu activation...
Results for model - Layers: 6, Units: 256, Dropout: 0.6, Activation: leakyrelu
Accuracy: 0.8368, AUC-PR: 0.9122, Precision: 0.7782, Recall: 0.9421

Training model with 3 layers, 256 units, 0.5 dropout, and gelu activation...
Results for model - Layers: 3, Units: 256, Dropout: 0.5, Activation: gelu
Accuracy: 0.8378, AUC-PR: 0.9372, Precision: 0.7713, Recall: 0.9602

Training model with 5 layers, 32 units, 0.5 dropout, and softplus activation...
Results for model - Layers: 5, Units: 32, Dropout: 0.5, Activation: softplus
Accuracy: 0.8466, A

Results for model - Layers: 6, Units: 64, Dropout: 0.0, Activation: relu
Accuracy: 0.8797, AUC-PR: 0.9027, Precision: 0.8505, Recall: 0.9215

Training model with 6 layers, 64 units, 0.5 dropout, and softplus activation...
Results for model - Layers: 6, Units: 64, Dropout: 0.5, Activation: softplus
Accuracy: 0.8397, AUC-PR: 0.9576, Precision: 0.7829, Recall: 0.9401

Training model with 5 layers, 256 units, 0.0 dropout, and relu activation...
Results for model - Layers: 5, Units: 256, Dropout: 0.0, Activation: relu
Accuracy: 0.8621, AUC-PR: 0.8713, Precision: 0.8653, Recall: 0.8576

Training model with 5 layers, 128 units, 0.5 dropout, and relu activation...
Results for model - Layers: 5, Units: 128, Dropout: 0.5, Activation: relu
Accuracy: 0.8603, AUC-PR: 0.9346, Precision: 0.8216, Recall: 0.9205

Training model with 5 layers, 64 units, 0.5 dropout, and leakyrelu activation...
Results for model - Layers: 5, Units: 64, Dropout: 0.5, Activation: leakyrelu
Accuracy: 0.8360, AUC-PR: 0.9139,

In [ ]:
dl_results_sorted = dl_results.sort_values('accuracy', ascending=False)

In [ ]:
best_model = dl_results_sorted.iloc[0]

In [ ]:
print("Best model configuration:")
print(f"Layers: {best_model['layers']}")
print(f"Units: {best_model['units']}")
print(f"Dropout: {best_model['dropout']}")
print(f"Activation Function: {best_model['activation_function']}")
print(f"Accuracy: {best_model['accuracy']:.4f}")
print(f"AUC-PR: {best_model['auc_pr']:.4f}")
print(f"Precision: {best_model['precision']:.4f}")
print(f"Recall: {best_model['recall']:.4f}")

In [ ]:
best_model_path = f"temp_model_layers{best_model['layers']}_units{best_model['units']}_dropout{best_model['dropout']}_{best_model['activation_function']}.h5"

# Recreate the best model architecture
best_model_architecture = Sequential()
best_model_architecture.add(Input(shape=(X_train_scaled.shape[1],)))
best_model_architecture.add(Dense(best_model['units']))
if best_model['activation_function'] == 'leakyrelu':
    best_model_architecture.add(LeakyReLU(alpha=0.01))
else:
    best_model_architecture.add(Activation(best_model['activation_function']))
best_model_architecture.add(Dropout(best_model['dropout']))

for _ in range(1, best_model['layers']):
    best_model_architecture.add(Dense(best_model['units']))
    if best_model['activation_function'] == 'leakyrelu':
        best_model_architecture.add(LeakyReLU(alpha=0.01))
    else:
        best_model_architecture.add(Activation(best_model['activation_function']))
    best_model_architecture.add(Dropout(best_model['dropout']))

best_model_architecture.add(Dense(1, activation='sigmoid'))

# Compile the model
best_model_architecture.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', AUC(curve='PR', name='auc_pr'), Precision(name='precision'), Recall(name='recall')])

# Load the best weights
best_model_architecture.load_weights(best_model_path)

In [ ]:
evaluation = best_model_architecture.evaluate(X_valid_scaled, y_valid, verbose=0)
print("\nBest model performance on validation set:")
print(f"Loss: {evaluation[0]:.4f}")
print(f"Accuracy: {evaluation[1]:.4f}")
print(f"AUC-PR: {evaluation[2]:.4f}")
print(f"Precision: {evaluation[3]:.4f}")
print(f"Recall: {evaluation[4]:.4f}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, average_precision_score, precision_score, recall_score

# Assuming X_train_scaled, y_train, X_valid_scaled, y_valid are numpy arrays
# Convert data to PyTorch tensors
X_train = torch.FloatTensor(X_train_scaled)
y_train = torch.FloatTensor(y_train)
X_valid = torch.FloatTensor(X_valid_scaled)
y_valid = torch.FloatTensor(y_valid)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

# Define the model architecture
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, activation):
        super(NeuralNet, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_size, hidden_size))
        
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(hidden_size, hidden_size))
        
        self.dropout = nn.Dropout(dropout)
        
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'leakyrelu':
            self.activation = nn.LeakyReLU(0.01)
        elif activation == 'gelu':
            self.activation = nn.GELU()
        else:  # softplus
            self.activation = nn.Softplus()
        
        self.output = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        for layer in self.layers:
            x = self.dropout(self.activation(layer(x)))
        return self.sigmoid(self.output(x))

# Set the best hyperparameters
best_layers = 5
best_units = 128
best_dropout = 0.3
best_activation = 'relu'

# Initialize the model
input_size = X_train.shape[1]
model = NeuralNet(input_size, best_units, best_layers, best_dropout, best_activation)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 20
best_val_accuracy = 0
best_model_path = f"best_model_layers{best_layers}_units{best_units}_dropout{best_dropout}_{best_activation}.pth"

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X).squeeze()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_valid).squeeze()
        val_loss = criterion(val_outputs, y_valid)
        val_preds = (val_outputs > 0.5).float()
        val_accuracy = accuracy_score(y_valid, val_preds)
        val_auc_pr = average_precision_score(y_valid, val_outputs)
        val_precision = precision_score(y_valid, val_preds)
        val_recall = recall_score(y_valid, val_preds)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}")
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), best_model_path)

# Load the best model
model.load_state_dict(torch.load(best_model_path))

# Final evaluation
model.eval()
with torch.no_grad():
    val_outputs = model(X_valid).squeeze()
    val_loss = criterion(val_outputs, y_valid)
    val_preds = (val_outputs > 0.5).float()
    val_accuracy = accuracy_score(y_valid, val_preds)
    val_auc_pr = average_precision_score(y_valid, val_outputs)
    val_precision = precision_score(y_valid, val_preds)
    val_recall = recall_score(y_valid, val_preds)

print(f"\nFinal model performance on validation set:")
print(f"Loss: {val_loss.item():.4f}")
print(f"Accuracy: {val_accuracy:.4f}")
print(f"AUC-PR: {val_auc_pr:.4f}")
print(f"Precision: {val_precision:.4f}")
print(f"Recall: {val_recall:.4f}")